In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 15.1 MB/s eta 0:00:00


In [ ]:
!pip install deep_sort_realtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 42.9 MB/s eta 0:00:00


In [4]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

# Load YOLO model
model = YOLO('yolov8n.pt')  # Load YOLOv8 pre-trained model

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=2, nn_budget=100)

# Load the video file
video_path = '/content/drive/MyDrive/detection_tracking_counting/video_6.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = '/content/drive/MyDrive/detection_tracking_counting/output_full_video_6_cars.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Set to keep track of unique car IDs
unique_car_ids = set()

while cap.isOpened():  # Process the entire video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection using YOLO
    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes.data:  # Extract bounding box info
            x1, y1, x2, y2, conf, cls = map(float, box[:6])
            cls = int(cls)  # Class ID
            if cls == 2:  # Only process cars (Class ID 2)
                detections.append([[x1, y1, x2, y2], conf])  # Format for DeepSORT

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = map(int, track.to_ltwh())
        track_id = track.track_id

        # Add the track ID to unique car IDs
        unique_car_ids.add(track_id)

        # Draw bounding boxes and track ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Green box for cars
        label = f'Car ID {track_id}'
        cv2.putText(frame, label, (x1, y1 - 15), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, (0, 0, 255), 3)  # Red text, larger font size

    # Display total unique cars on the frame
    total_cars_text = f'Total Unique Cars: {len(unique_car_ids)}'
    cv2.putText(frame, total_cars_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1.5, (255, 255, 0), 3)  # Cyan text, top-left corner

    # Write the frame to output
    out.write(frame)

cap.release()
out.release()

print("Detection, tracking, and counting complete. Output saved to:", output_path)



0: 384x640 9 persons, 9 cars, 3 motorcycles, 1 stop sign, 1 backpack, 155.5ms
Speed: 6.3ms preprocess, 155.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 9 cars, 4 motorcycles, 1 stop sign, 1 backpack, 138.1ms
Speed: 4.0ms preprocess, 138.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 9 cars, 4 motorcycles, 1 stop sign, 1 backpack, 129.0ms
Speed: 5.7ms preprocess, 129.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 6 cars, 4 motorcycles, 1 stop sign, 1 backpack, 129.5ms
Speed: 4.7ms preprocess, 129.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 6 cars, 4 motorcycles, 1 backpack, 133.6ms
Speed: 3.9ms preprocess, 133.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 5 cars, 4 motorcycles, 1 backpack, 128.0ms
Speed: 4.0ms preprocess, 128.0ms inference, 1.2ms postproc